In [1]:
import pandas as pd
import numpy as np
import timeit

In [2]:
#read in the data
df = pd.read_parquet("../../data/input/trip_data_sample.parquet")

In [3]:
df

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,...,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,dropoff_census_tract,company
15682557,ecb7736258ccc9885843ab4812403c02d287ea5a,e5c165fcc578673a02367d8d379846b48b00fbacf7f1b4...,2013-08-13 10:30:00,2013-08-13 10:30:00,360.0,1.3,6.45,0.00,0.0,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None
12211112,95e0d77a57c76e5a885a03a29c61396115229749,2f5a8b67f988625ed892f9f87d48255861bae9beaeb573...,2013-06-28 18:15:00,2013-06-28 18:30:00,600.0,1.2,6.85,0.00,0.0,1.0,...,8.0,41.898332,-87.620763,"{'coordinates': [-87.6207628651, 41.8983317935...",8.0,41.892042,-87.631864,"{'coordinates': [-87.6318639497, 41.8920421365...",1.703108e+10,None
2361088,12033fd51c7710c80265e2df044655bd6fb1bd24,36c49e0df2325e7e57fc13127f76b6962b2697184dab38...,2013-02-14 00:45:00,2013-02-14 01:00:00,780.0,1.5,15.85,0.00,0.0,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None
2479418,e513f8f342774d9f5f64c080bef0af84ff091049,2f1df8a630d47fa448585b72a26287f920aa3349fb90e6...,2013-02-15 20:00:00,2013-02-15 20:30:00,1140.0,0.1,11.25,0.00,0.0,1.0,...,8.0,41.892073,-87.628874,"{'coordinates': [-87.6288741572, 41.8920726347...",28.0,41.878667,-87.671654,"{'coordinates': [-87.6716536214, 41.8786674201...",1.703184e+10,Taxi Affiliation Services
7723418,fad03f3d5b792c0896b60934a3aaf7a9cffa8bf5,4956d3181718741f82386d54c22d3288ebaba0cfff19c6...,2013-04-30 10:00:00,2013-04-30 10:15:00,660.0,0.1,9.45,0.00,0.0,0.0,...,6.0,41.944227,-87.655998,"{'coordinates': [-87.6559981815, 41.9442266014...",4.0,41.975171,-87.687516,"{'coordinates': [-87.68751551520002, 41.975170...",NaN,Taxi Affiliation Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13979084,013b1cb5cfab75177de7ac4f39c3a8a23073cca3,2e9bb6bc95512aed0405c8f63b7712de3526a75eb144ca...,2013-07-21 18:45:00,2013-07-21 19:00:00,600.0,1.9,8.05,0.00,0.0,1.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None
26090563,0aca0c9dbb6da97c6e855f9cab731f26eeadf553,de22250c7bf3b590388f1bd1c2f7ca36c525efbe4438bf...,2013-12-16 21:00:00,2013-12-16 21:00:00,625.0,4.2,11.45,3.11,NaN,1.0,...,8.0,41.892508,-87.626215,"{'coordinates': [-87.6262149064, 41.8925077809...",6.0,41.942692,-87.651771,"{'coordinates': [-87.6517705068, 41.9426918444...",1.703106e+10,American United
25348687,c95f9c1b0358938d9c21b5ddf879a424e972562f,b5c33c47b123870895f56c87207368e3891752795531cc...,2013-12-09 10:00:00,2013-12-09 10:15:00,180.0,0.6,4.65,0.00,0.0,0.0,...,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None
6902865,d164c5fe227e869b3328d67ed9b86e97fb366627,5b5fd5f13833d7b8d871d1573fb8ef3c1b39417d1802a5...,2013-04-18 22:30:00,2013-04-18 23:00:00,2040.0,1.4,47.25,0.00,0.0,3.0,...,76.0,41.980264,-87.913625,"{'coordinates': [-87.913624596, 41.9802643146]...",39.0,41.808916,-87.596183,"{'coordinates': [-87.5961833442, 41.8089162826...",NaN,Taxi Affiliation Services


In [4]:
df.dtypes

trip_id                               object
taxi_id                               object
trip_start_timestamp          datetime64[ns]
trip_end_timestamp            datetime64[ns]
trip_seconds                         float64
trip_miles                           float64
fare                                 float64
tips                                 float64
tolls                                float64
extras                               float64
trip_total                           float64
payment_type                          object
pickup_census_tract                  float64
pickup_community_area                float64
pickup_centroid_latitude             float64
pickup_centroid_longitude            float64
pickup_centroid_location              object
dropoff_community_area               float64
dropoff_centroid_latitude            float64
dropoff_centroid_longitude           float64
dropoff_centroid_location             object
dropoff_census_tract                 float64
company   

In [5]:
def add_id(df):
    # new trip id 
    df["trip_id"] = df.index
    # merge the new Taxi Id
    a=df["taxi_id"].unique()
    a=np.array(a)
    a=pd.DataFrame(a)
    a["id"] =a.index
    a.columns.values[0] = "taxi_id"
    df = pd.merge(df, a, on="taxi_id", how='left') #merge id with new
    df=df.drop(["taxi_id"], axis=1)
    df["taxi_id"] = df["id"]
    df=df.drop(["id"], axis=1)
    
    return df
    

# Notebook for Performance Tests

### Testing the change of Trip Id (char) to int64 and Taxi ID (char) to Int64

In [6]:
%%timeit
df.groupby(["tolls"])["trip_id"].count()

529 ms ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
df.groupby(["tolls"])["taxi_id"].count()

506 ms ± 48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Changing IDs

In [8]:
df=add_id(df)

In [9]:
%%timeit
df.groupby(["tolls"])["trip_id"].count()

145 ms ± 9.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
df.groupby(["tolls"])["taxi_id"].count()

155 ms ± 8.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,dropoff_census_tract,company,taxi_id
0,15682557,2013-08-13 10:30:00,2013-08-13 10:30:00,360.0,1.3,6.45,0.00,0.0,0.0,6.45,...,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,0
1,12211112,2013-06-28 18:15:00,2013-06-28 18:30:00,600.0,1.2,6.85,0.00,0.0,1.0,7.85,...,41.898332,-87.620763,"{'coordinates': [-87.6207628651, 41.8983317935...",8.0,41.892042,-87.631864,"{'coordinates': [-87.6318639497, 41.8920421365...",1.703108e+10,None,1
2,2361088,2013-02-14 00:45:00,2013-02-14 01:00:00,780.0,1.5,15.85,0.00,0.0,0.0,15.85,...,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,2
3,2479418,2013-02-15 20:00:00,2013-02-15 20:30:00,1140.0,0.1,11.25,0.00,0.0,1.0,12.25,...,41.892073,-87.628874,"{'coordinates': [-87.6288741572, 41.8920726347...",28.0,41.878667,-87.671654,"{'coordinates': [-87.6716536214, 41.8786674201...",1.703184e+10,Taxi Affiliation Services,3
4,7723418,2013-04-30 10:00:00,2013-04-30 10:15:00,660.0,0.1,9.45,0.00,0.0,0.0,9.45,...,41.944227,-87.655998,"{'coordinates': [-87.6559981815, 41.9442266014...",4.0,41.975171,-87.687516,"{'coordinates': [-87.68751551520002, 41.975170...",NaN,Taxi Affiliation Services,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515086,13979084,2013-07-21 18:45:00,2013-07-21 19:00:00,600.0,1.9,8.05,0.00,0.0,1.0,9.05,...,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,2496
4515087,26090563,2013-12-16 21:00:00,2013-12-16 21:00:00,625.0,4.2,11.45,3.11,NaN,1.0,15.56,...,41.892508,-87.626215,"{'coordinates': [-87.6262149064, 41.8925077809...",6.0,41.942692,-87.651771,"{'coordinates': [-87.6517705068, 41.9426918444...",1.703106e+10,American United,3039
4515088,25348687,2013-12-09 10:00:00,2013-12-09 10:15:00,180.0,0.6,4.65,0.00,0.0,0.0,4.65,...,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,202
4515089,6902865,2013-04-18 22:30:00,2013-04-18 23:00:00,2040.0,1.4,47.25,0.00,0.0,3.0,50.25,...,41.980264,-87.913625,"{'coordinates': [-87.913624596, 41.9802643146]...",39.0,41.808916,-87.596183,"{'coordinates': [-87.5961833442, 41.8089162826...",NaN,Taxi Affiliation Services,2781


In [13]:
df.dtypes

trip_id                                int64
trip_start_timestamp          datetime64[ns]
trip_end_timestamp            datetime64[ns]
trip_seconds                         float64
trip_miles                           float64
fare                                 float64
tips                                 float64
tolls                                float64
extras                               float64
trip_total                           float64
payment_type                          object
pickup_census_tract                  float64
pickup_community_area                float64
pickup_centroid_latitude             float64
pickup_centroid_longitude            float64
pickup_centroid_location              object
dropoff_community_area               float64
dropoff_centroid_latitude            float64
dropoff_centroid_longitude           float64
dropoff_centroid_location             object
dropoff_census_tract                 float64
company                               object
taxi_id   